In [5]:
!pip install torchmetrics
!pip install tqdm
!pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as  mape
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR, LinearSVR

from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchmetrics import MeanSquaredLogError
from tqdm import tqdm
from torch.optim import AdamW

import tensorflow as tf
from tensorflow import keras as ks
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
import seaborn as sns


In [7]:
from preprocess import EstateData

dataset = EstateData("final.csv")

In [8]:
dataset.split_data(stratify=True)

In [9]:
dataset.preprocess(tukey=True)

Start Process!
Fill missing values: Done
Remove outlier: Done
Scale features: Done


In [10]:
train = dataset.train
test = dataset.test

In [11]:
print("train size {}".format(train.shape))
print("test size {}".format(test.shape))

train size (31531, 13)
test size (9278, 13)


In [12]:
train, dev = train_test_split(train, test_size=0.1)

In [13]:
train = pd.get_dummies(train)
dev = pd.get_dummies(dev)
test = pd.get_dummies(test)

In [14]:
features = train.columns.tolist()
features.remove("Price")
features.remove("PricePerM2")

In [18]:
model = ks.Sequential()
model.add(ks.layers.Dense(100, input_dim = 64, activation = 'relu', name = 'input'))
model.add(ks.layers.Dropout(0.25))
model.add(ks.layers.Dense(100, activation = 'relu', name = 'hidden1'))
model.add(ks.layers.Dropout(0.15))
model.add(ks.layers.Dense(100, activation = 'relu', name = 'hidden2'))
model.add(ks.layers.Dropout(0.15))
model.add(ks.layers.Dense(100, activation = 'relu', name = 'hidden3'))
model.add(ks.layers.Dropout(0.15))
model.add(ks.layers.Dense(1, name = 'output'))
opt = ks.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = opt, 
              loss = tf.keras.losses.MeanSquaredLogarithmicError(), 
              metrics = ['mean_absolute_percentage_error'])
train_history = model.fit(train[features], train["PricePerM2"], batch_size = 2048, epochs = 1000, validation_split = 0.1)
ann_train_res = model.evaluate(dev[features], dev["PricePerM2"])
ann_test_res = model.evaluate(test[features], test["PricePerM2"])
print("Train:  Loss = %f, Mean_absolute percentage_error = %f" %(ann_train_res[0], ann_train_res[1]/100))
print("Test:  Loss = %f, Mean_absolute_perntage_error = %f" %(ann_test_res[0], ann_test_res[1]/100))

2023-02-09 15:11:42.769629: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 13075968 exceeds 10% of free system memory.


Epoch 1/1000
13/13 [==============================] - 13s 236ms/step - loss: 8.1435 - mean_absolute_percentage_error: 304.4926 - val_loss: 3.5942 - val_mean_absolute_percentage_error: 763.4014
Epoch 2/1000
13/13 [==============================] - 1s 104ms/step - loss: 2.5100 - mean_absolute_percentage_error: 481.5879 - val_loss: 1.1614 - val_mean_absolute_percentage_error: 129.8337
Epoch 3/1000
13/13 [==============================] - 1s 107ms/step - loss: 1.2294 - mean_absolute_percentage_error: 100.8238 - val_loss: 0.9624 - val_mean_absolute_percentage_error: 91.8390
Epoch 4/1000
13/13 [==============================] - 1s 113ms/step - loss: 0.9251 - mean_absolute_percentage_error: 117.3004 - val_loss: 0.7841 - val_mean_absolute_percentage_error: 113.8351
Epoch 5/1000
13/13 [==============================] - 1s 106ms/step - loss: 0.7760 - mean_absolute_percentage_error: 105.2450 - val_loss: 0.6666 - val_mean_absolute_percentage_error: 86.4480
Epoch 6/1000
13/13 [=====================

In [19]:
pred = model.predict(test[features])
print(mape(test["PricePerM2"],pred ))


290/290 [==============================] - 0s 1ms/step
0.5644196438650978
